In [1]:
import polars as pl

In [29]:
random_df = pl.read_csv('~/sports-language-in-politics/data/processed/random_sample.csv')
sports_df = pl.read_csv('~/sports-language-in-politics/data/processed/sports_sample.csv')

sports_subs = set(sports_df['subreddit'].unique().to_list())
random_subs = set(random_df['subreddit'].unique().to_list())

In [31]:
random_df_no_sports = random_df.filter(~pl.col('subreddit').is_in(sports_subs))

In [32]:
random_df_no_sports.write_csv('/users/ujan/sports-language-in-politics/data/processed/random_sample_no_sports.csv', separator=",")

#### remove gaming subs

In [2]:
random_df = pl.read_csv('~/sports-language-in-politics/data/processed/random_sample_no_sports.csv')

In [3]:
random_df.head()

body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,distinguished,author_flair_css_class,removal_reason
str,str,str,str,str,str,str,i64,str,str,str,i64,i64,i64,i64,str,str,str,str,str
"""anime avatar i…",null,null,null,"""GVBN""","""http://myanime…",null,1500892341,"""t5_2qh22""","""t3_6p7hsi""","""t3_6p7hsi""",31,1501516961,0,0,"""dkn63pv""","""anime""",null,"""MAL""",null
"""I say remove t…",null,null,null,"""h2ppyme""","""Estonia""",null,1501056473,"""t5_2qh4j""","""t3_6pmde2""","""t1_dkqhoal""",17,1501853060,0,0,"""dkqi2tv""","""europe""",null,"""ESTO""",null
"""Last time I re…",null,null,null,"""iscreamcoke""","""France""",null,1499804412,"""t5_2qh4j""","""t3_6moqbx""","""t1_dk35u69""",21,1501157745,0,0,"""dk3721q""","""europe""",null,"""FRAN""",null
"""Her name was C…",null,null,null,"""Robonator7of9""","""fa/tg/uy""",null,1499098546,"""t5_2qh4w""","""t3_6ky9ul""","""t1_djpzb0t""",77,1499870014,0,0,"""djq7kca""","""4chan""",null,"""""",null
"""Ah ben oui mai…",null,null,null,"""DrShadoko""","""Minitel""",null,1499440702,"""t5_2qhjz""","""t3_6lu5pb""","""t3_6lu5pb""",24,1501037227,0,0,"""djwlcaq""","""france""",null,"""minitel""",null


In [8]:
gaming_df = pl.read_csv('~/sports-language-in-politics/data/raw/gaming_subs.csv', truncate_ragged_lines=True)

In [21]:
gaming_subs = []
for i in range(len(gaming_df)):
    gaming_subs.append(gaming_df[i]['Name;Link'].to_list()[0].split('/r/')[-1])

In [24]:
len(random_df)

9280850

In [25]:
random_df_no_gaming = random_df.filter(~pl.col('subreddit').is_in(gaming_subs))

In [27]:
len(random_df_no_gaming)

8467892

In [29]:
random_df_no_gaming.write_csv('/users/ujan/sports-language-in-politics/data/processed/random_sample_no_sports.csv', separator=",")